In this notebook I will plot all the longidutinal data for each individual separately

In [1]:
import src.data.breathe_data as br
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import src.data.helpers as dh
from datetime import timedelta
import src.o2_fev1_analysis.smooth as smooth
import numpy as np

In [2]:
# df = br.build_O2_FEV1_FEF2575_df(meas_file=2, remove_nan=False)
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")
df["PEF (L/s)"] = df.PEF / 60

In [2]:
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA")
df["PEF (L/s)"] = df.PEF / 60

In [10]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,...,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,PEF (L/s),FEF2575%PEF,AR,IA,AR mean,IA mean
0,101,2021-05-25,1.68,98,1.17,227,1.69,53,Male,173.0,...,97.150104,46.813611,46.536607,100.874827,3.783333,30.925110,[2.01402681e-04 2.56395073e-04 3.28923089e-04 ...,[8.56146734e-01 1.33401262e-01 1.01430010e-02 ...,47.601368,0.654616
1,101,2021-05-26,1.65,98,1.06,236,1.69,53,Male,173.0,...,97.150104,46.813611,45.705597,100.874827,3.933333,26.949153,[2.01402681e-04 2.56395073e-04 3.28923089e-04 ...,[8.56146734e-01 1.33401262e-01 1.01430010e-02 ...,47.601368,0.654616
2,101,2021-05-27,1.69,98,1.12,183,1.69,53,Male,173.0,...,97.150104,46.813611,46.813611,100.874827,3.050000,36.721311,[2.01402681e-04 2.56395073e-04 3.28923089e-04 ...,[8.56146734e-01 1.33401262e-01 1.01430010e-02 ...,47.601368,0.654616
3,101,2021-05-28,1.67,98,1.08,175,1.69,53,Male,173.0,...,97.150104,46.813611,46.259604,100.874827,2.916667,37.028571,[2.01402681e-04 2.56395073e-04 3.28923089e-04 ...,[8.56146734e-01 1.33401262e-01 1.01430010e-02 ...,47.601368,0.654616
4,101,2021-05-29,1.69,98,1.16,171,1.76,53,Male,173.0,...,97.150104,48.752636,46.813611,100.874827,2.850000,40.701754,[3.21119510e-04 4.10972961e-04 5.29609732e-04 ...,[8.45297834e-01 1.42577868e-01 1.17271689e-02 ...,44.593062,0.667227


In [6]:
df1.to_excel(
    f"{dh.get_path_to_main()}ExcelFiles/BR/BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA_processed.xlsx",
    index=False,
)

In [7]:
def apply_new_smoothing(df, col, scale, shift):
    df[f"ec{col}_2"] = df[col]
    no_nan_mask = ~df[f"ec{col}_2"].isna()
    df.iloc[no_nan_mask] = smooth.identify_and_replace_outliers_up(
        df[no_nan_mask], f"ec{col}_2", scale=scale, shift=shift
    ).copy()
    # df[f"ec{col}_3"] = df[f"ec{col}_2"]
    df[f"ec{col}_2"][no_nan_mask] = smooth.smooth_vector(
        df[f"ec{col}_2"][no_nan_mask].to_numpy(), "max"
    )

    return df


df1 = (
    # df[df.ID == "104"]
    df.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEV1", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEF2575", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "PEF (L/s)", 3, 1))
    .reset_index(drop=True)
)


def plot_profile_for_ID(df_for_ID):
    df_for_ID = df_for_ID.reset_index()
    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.04,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["O2 Saturation"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=2, col=1)
    fig.add_trace(
        go.Scatter(y=df_for_ID.ecFEV1_2, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.FEF2575,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=3,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=3, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.ecFEF2575_2, x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["PEF (L/s)"],
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=4,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=4, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["ecPEF (L/s)_2"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=4,
        col=1,
    )
    fig.update_yaxes(
        title="SpO2<br>(%)",
        row=1,
        col=1,
        range=[
            np.nanmin(df["O2 Saturation"]) * 0.98,
            np.nanmax(df["O2 Saturation"]) * 1.02,
        ],
    )
    fig.update_yaxes(
        title="FEV1<br>(L)",
        row=2,
        col=1,
        range=[np.nanmin(df.FEV1) * 0.98, np.nanmax(df.FEV1) * 1.02],
    )
    fig.update_yaxes(
        title="FEF2575<br>(L/s)",
        row=3,
        col=1,
        range=[np.nanmin(df.FEF2575) * 0.98, np.nanmax(df.FEF2575) * 1.02],
    )
    fig.update_yaxes(
        title="PEF<br>(L/s)",
        row=4,
        col=1,
        range=[np.nanmin(df.PEF) / 60 * 0.98, np.nanmax(df.PEF) / 60 * 1.02],
    )
    fig.update_yaxes(nticks=10)
    fig.update_traces(marker=dict(size=3))
    title = f"Measures for ID {df_for_ID.ID[0]}"
    fig.update_layout(
        font=dict(size=8), showlegend=False, title=title, height=1000, width=1400
    )
    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/ID_profiles/{title}.pdf")
    return


df1.groupby(by="ID").apply(plot_profile_for_ID)

ID 221 - Outlier up for ecFEV1_2, day 2022-08-09: 3.03 > 2.45 and > 2.44 (mean=1.94,std=0.17), update to 2.06
ID 244 - Outlier up for ecFEV1_2, day 2022-08-27: 4.38 > 3.87 and > 3.61 (mean=3.11,std=0.26), update to 3.06
ID 104 - Outlier up for ecFEF2575_2, day 2023-07-26: 1.73 > 1.72 and > 1.67 (mean=1.17,std=0.18), update to 1.12
ID 123 - Outlier up for ecFEF2575_2, day 2022-01-04: 5.66 > 5.65 and > 5.21 (mean=4.71,std=0.31), update to 4.94
ID 123 - Outlier up for ecFEF2575_2, day 2022-11-27: 5.45 > 5.38 and > 5.14 (mean=4.64,std=0.25), update to 4.55
ID 172 - Outlier up for ecFEF2575_2, day 2022-08-10: 4.23 > 4.05 and > 3.46 (mean=2.96,std=0.37), update to 2.92
ID 203 - Outlier up for ecFEF2575_2, day 2022-09-21: 2.25 > 1.98 and > 1.75 (mean=1.25,std=0.24), update to 1.20
ID 221 - Outlier up for ecFEF2575_2, day 2022-08-09: 3.46 > 2.96 and > 2.78 (mean=2.28,std=0.23), update to 2.44
ID 240 - Outlier up for ecFEF2575_2, day 2023-01-17: 4.64 > 4.43 and > 2.63 (mean=2.13,std=0.77), upda

""


In [5]:
df1[df1.FEV1 > 2.3]

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,ecFEF2575,ecPEF,Age,...,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,PEF (L/s),ecFEV1_2,ecFEV1_3,ecFEF2575_2,ecFEF2575_3,ecPEF (L/s)_2,ecPEF (L/s)_3
337,104,2023-10-29,2.74,97.0,NaN,NaN,2.74,NaN,NaN,25,...,109.591681,109.591681,98.467995,NaN,2.74,2.74,NaN,NaN,NaN,NaN
